<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
wire_radius = 25e-6; distance = 50e-6; wire_current = 10; mu_freeSpace = 4*np.pi*1e-7; chi = -2.2e-5
outer_linear_density = 7; inner_linear_density = 7

outer_points = np.linspace(wire_radius, distance, outer_linear_density); inner_points = np.linspace(0,wire_radius,inner_linear_density,endpoint=False)

In [ ]:
x_linear_point = np.concatenate((-outer_points[::-1],-inner_points[::-1][:-1],inner_points,outer_points))
y_linear_point = x_linear_point

In [ ]:
x_linear_point

In [ ]:
# Important Modfications for calculating phi from (x,y)
(np.arctan(-1/-1)+np.pi)*180/np.pi,(2*np.pi+np.arctan(-1/1))*180/np.pi,(np.arctan(1/-1)+np.pi)*180/np.pi

In [ ]:
def MagField(current,mu,chi,wire_radius,x,y):
  field = np.ones((x.shape[0],y.shape[0]))
  field_xy = np.ones((x.shape[0],y.shape[0],2))
  quiver_xy = np.ones((x.shape[0],y.shape[0],2))
  for i in range(x.shape[0]):
    for j in range(y.shape[0]):
      radius = np.sqrt(x[i]**2+y[j]**2)
      if radius<=wire_radius:
        field[i,j] = None
        field_xy[i,j] = np.array([None,None])
        quiver_xy[i,j] = np.array([x[i],y[j]])
      else:
        if x[i]==0:
          angle = np.arctan(y[j]*np.inf)
        # as shown above
        elif x[i]<0 and y[j]<0:
          angle = np.arctan(y[j]/x[i])+np.pi
        elif x[i]>0 and y[j]<0:
          angle = np.arctan(y[j]/x[i])+2*np.pi
        elif x[i]<0 and y[j]>0:
          angle = np.arctan(y[j]/x[i])+np.pi
        elif x[i]>0 and y[j]>0:
          angle = np.arctan(y[j]/x[i])

        if y[j]<(-wire_radius):
          mu_below = mu*(1+chi)
          field[i,j] = (mu_below*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy[i,j] = np.array([field[i,j]*np.cos(angle), field[i,j]*np.sin(angle)])
          quiver_xy[i,j] = np.array([x[i],y[j]])
        else:
          field[i,j] = (mu*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy[i,j] = np.array([field[i,j]*np.cos(angle), field[i,j]*np.sin(angle)])
          quiver_xy[i,j] = np.array([x[i],y[j]])
  return field,field_xy,quiver_xy

In [ ]:
mag_field,field_xy,quiver_xy = MagField(current=wire_current,mu=mu_freeSpace,chi=chi,wire_radius=wire_radius,x=x_linear_point,y=y_linear_point)

In [ ]:
fig = go.Figure()
# Heatmap
fig.add_trace(go.Contour(x=x_linear_point,y=y_linear_point,z=mag_field,contours_coloring='heatmap',
                                connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
fig.update_traces(line_width=0)

# Quiver
for i in range(1,quiver_xy.shape[0]-1,2):
  x = quiver_xy[i,1:-1:2,0]; y = quiver_xy[i,1:-1:2,1]
  u = -field_xy[i,1:-1:2,1]; v = field_xy[i,1:-1:2,0]
  quiver_trace = ff.create_quiver(x, y, u, v, scale=0.00006, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black')).data[0]
  fig.add_trace(quiver_trace)

# Inner Circle
fig.add_trace(go.Scatter(x=wire_radius*np.cos(np.linspace(0,2*np.pi,1000)), y=wire_radius*np.sin(np.linspace(0,2*np.pi,1000)),
                         mode='lines', line=dict(color='yellow', width=0.7),fill = 'tonexty',fillcolor = 'rgba(127, 0, 255, 0.4)'))

# Outer Circles
[fig.add_trace(go.Scatter(x=radius*np.cos(np.linspace(0,2*np.pi,1000)), y=radius*np.sin(np.linspace(0,2*np.pi,1000)),
                         mode='lines', line=dict(color='yellow', width=0.7))) for radius in np.linspace(wire_radius,distance,outer_linear_density)]

# Other Ornamentations
fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker_size=15, marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=x_linear_point, y=-wire_radius*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
fig.add_trace(go.Scatter(x=x_linear_point, y=-y_linear_point[-1]*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='grey'),
                         fill='tonexty',fillcolor = 'rgba(255, 255, 255, 0.5)'))

fig.update_layout(template=fig_template, title = None,width=800, height=700)
fig.update_xaxes(title = 'Distance',range = [x_linear_point[0],x_linear_point[-1]])
fig.update_yaxes(title = 'Distance', range = [y_linear_point[0],y_linear_point[-1]])
fig.update_layout(showlegend=False)

fig.show()